In [1]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

client = Client(n_workers=12,
                threads_per_worker=2,
                memory_limit='4GB')

pgb = ProgressBar()
pgb.register()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

df = dd.read_parquet('../../data/series_train_agg.parquet')

In [3]:
df.head()

step         X         Y  \
5_min_chunk id       relative_date_PCIAT                                
86.0        9e052412 75.0                  9377.5 -0.813552 -0.103173   
154.0       9e052412 75.0                 13457.5 -0.093014  0.033776   
23.0        9e052412 76.0                 22877.5 -0.679405  0.016084   
                     77.0                 40157.5 -0.973359 -0.185620   
118.0       9e052412 77.0                 45857.5 -0.496541  0.468611   

                                                 Z      enmo     anglez  \
5_min_chunk id       relative_date_PCIAT                                  
86.0        9e052412 75.0                 0.441742  0.003841  26.689215   
154.0       9e052412 75.0                -0.629198  0.137143 -45.763802   
23.0        9e052412 76.0                -0.730851  0.000508 -47.104785   
                     77.0                -0.071164  0.000018  -4.110943   
118.0       9e052412 77.0                -0.393024  0.056895 -27.909910   

                                          non-wear_flag        light  \
5_min_chunk id       relative_date_PCIAT                               
86.0        9e052412 75.0                           0.0     0.856442   
154.0       9e052412 75.0                           0.0  2384.940104   
23.0        9e052412 76.0                           0.0     1.597303   
                     77.0                           0.0    19.026412   
118.0       9e052412 77.0                           0.0     5.952778   

                                          battery_voltage   time_of_day  \
5_min_chunk id       relative_date_PCIAT                                  
86.0        9e052412 75.0                     4158.000000  2.594750e+13   
154.0       9e052412 75.0                     4149.433333  4.634750e+13   
23.0        9e052412 76.0                     4126.000000  7.047500e+12   
                     77.0                     4087.000000  7.047500e+12   
118.0       9e052412 77.0                     4078.200000  3.554750e+13   

                                          weekday  quarter  count  
5_min_chunk id       relative_date_PCIAT                           
86.0        9e052412 75.0                     2.0      3.0     60  
154.0       9e052412 75.0                     2.0      3.0     60  
23.0        9e052412 76.0                     3.0      3.0     60  
                     77.0                     4.0      3.0     60  
118.0       9e052412 77.0                     4.0      3.0     60

In [ ]:
def compute_quantiles(df):
    return pd.Series({
        **{
            col: np.quantile(df[col], [0, 0.25, 0.5, 0.75, 1])
            for col in df.columns if col not in ['id', '5_min_chunk', 'time_of_day', 'weekday', 'quarter', 'count']
        },
        "count": len(df)
    })

# meta = pd.DataFrame(
#     np.zeros((5, )),
#     columns=numeric_cols
# )

agged = df.groupby(['id', '5_min_chunk'], observed=True).apply(compute_quantiles)

/tmp/ipykernel_928389/2665491164.py:12: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  agged = df.groupby(['id', '5_min_chunk'], observed=True).apply(compute_quantiles)


In [7]:
(agged['count'] > 1).sum().compute()

KeyboardInterrupt: 